In [1]:
import numpy as np
import pandas as pd 
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten


import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

In [3]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [4]:
one_hot_labels = np.asarray(one_hot)

In [5]:
def read_and_resize_images(df,nrow=224,ncol=224,channels=3):
    from scipy import misc
    i = 0
    x = []
    y = []
    for f, breed in tqdm(df.values):
        img = misc.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x.append(misc.imresize(img, (nrow, ncol,channels)))
        y.append(label)
        i += 1
    return x,y

In [6]:
x_train,y_train = read_and_resize_images(df_train)

  0%|                                                                                        | 0/10222 [00:00<?, ?it/s]C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if __name__ == '__main__':
100%|████████████████████████████████████████████████████████████████████████████| 10222/10222 [01:49<00:00, 93.11it/s]


In [7]:
def read_and_resize_images_test(df,nrow=224,ncol=224,channels=3):
    from scipy import misc
    x = []
    for f in tqdm(df['id'].values):
        img = misc.imread('../input/test/{}.jpg'.format(f))
        x.append(misc.imresize(img, (nrow, ncol,channels)))
    return x

In [9]:
x_test = read_and_resize_images_test(df_test)

  0%|                                                                                        | 0/10357 [00:00<?, ?it/s]C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
100%|████████████████████████████████████████████████████████████████████████████| 10357/10357 [03:00<00:00, 57.37it/s]


In [10]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

MemoryError: 

In [11]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 224, 224, 3)
(10222, 120)


AttributeError: 'list' object has no attribute 'shape'

In [12]:
num_class = y_train_raw.shape[1]

In [13]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)

In [15]:
base_model = VGG19(#weights='imagenet',
    weights = None, include_top=False, input_shape=(224, 224, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid), verbose=1)

Train on 7155 samples, validate on 3067 samples
Epoch 1/1


In [ ]:
preds = model.predict(x_test, verbose=1)

In [ ]:
sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)